In [1]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v1/codes/model


In [2]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v1/codes


In [3]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v1


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb_o
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [5]:
def preprocessing():
    df = pd.read_csv('data/marge/marge.csv', index_col=0).reset_index(drop = True)
    M_TEAM_NAMES = pd.read_csv('data/other/TEAM_NAMES.csv').values.tolist()
    M_COACH_LIST = pd.read_csv('data/other/COACH_LIST.csv').columns.tolist()
    
    #     df = df[df['年月日'] < yyyyMMdd]
    df = df.dropna().reset_index(drop = True)
    
    # ○○、○○成功率　をまとめる
    h_a = ['H_', 'A_']
    par_calumns = ['パス', 'クロス', 'スローイン', 'ドリブル', 'タックル']
    for head in h_a:
        for col in par_calumns:
            df[head+'成功した'+col] = df[head + col] * df[head + col + '成功率']/100
            df[head+'失敗した'+col] = df[head + col] - df[head+'成功した'+col]
            df.drop(columns = [head + col], inplace = True)
            df.drop(columns = [head + col + '成功率'], inplace = True)
    
    hour_list = []
    for i, row in df.iterrows():
        hour_list += [row['キックオフ時刻'][:2]]
    df_tmp = pd.DataFrame(hour_list, columns = ['キックオフ時'])
    df = pd.concat([df, df_tmp],axis=1)
    df.drop(columns = ['キックオフ時刻'], inplace = True)
    
    hare_list, ame_list, kumori_list, okunai_list = [], [], [], []
    j1_list, j2_list, j3_list = [], [], []
    h_df_list, h_mf_list, h_fw_list = [], [], []
    a_df_list, a_mf_list, a_fw_list = [], [], []
    h_rank_list, a_rank_list = [], []
    for i, row in df.iterrows():
        whether = row['天候']
        category = row['カテゴリ']

        H_positions = ''
        A_positions = ''
        for i in range(1, 12):
            H_positions += row['H_ポジション' + str(i)]
            A_positions += row['A_ポジション' + str(i)]

        h_df_list += [H_positions.count('DF')]
        h_mf_list += [H_positions.count('MF')]
        h_fw_list += [H_positions.count('FW')]
        a_df_list += [A_positions.count('DF')]
        a_mf_list += [A_positions.count('MF')]
        a_fw_list += [A_positions.count('FW')]
        
        h_rank_list += [str(row['H_順位']).replace('※', '')]
        a_rank_list += [str(row['A_順位']).replace('※', '')]
        
        if category == 'J1':
            j1_list += [1]
            j2_list += [0]
            j3_list += [0]
        elif category == 'J2':
            j1_list += [0]
            j2_list += [1]
            j3_list += [0]
        else:
            j1_list += [0]
            j2_list += [0]
            j3_list += [1]

        if 0 < whether.count('晴'):
            hare_list += [1]
        else:
            hare_list += [0]
        if 0 < whether.count('雨'):
            ame_list += [1]
        else:
            ame_list += [0]
        if 0 < whether.count('曇'):
            kumori_list += [1]
        else:
            kumori_list += [0]
        if 0 < whether.count('室内'):
            okunai_list += [1]
        else:
            okunai_list += [0]

    df_tmp1 = pd.DataFrame(hare_list, columns = ['晴_flg'])
    df_tmp2 = pd.DataFrame(ame_list, columns = ['雨_flg'])
    df_tmp3 = pd.DataFrame(kumori_list, columns = ['曇_flg'])
    df_tmp4 = pd.DataFrame(okunai_list, columns = ['室内_flg'])
    df_tmp5 = pd.DataFrame(j1_list, columns = ['J1_flg'])
    df_tmp6 = pd.DataFrame(j2_list, columns = ['J2_flg'])
    df_tmp7 = pd.DataFrame(j3_list, columns = ['J3_flg'])
    df_tmp8 = pd.DataFrame(h_df_list, columns = ['H_DF'])
    df_tmp9 = pd.DataFrame(h_mf_list, columns = ['H_MF'])
    df_tmp10 = pd.DataFrame(h_fw_list, columns = ['H_FW'])
    df_tmp11 = pd.DataFrame(a_df_list, columns = ['A_DF'])
    df_tmp12 = pd.DataFrame(a_mf_list, columns = ['A_MF'])
    df_tmp13 = pd.DataFrame(a_fw_list, columns = ['A_FW'])
    df_tmp14 = pd.DataFrame(h_rank_list, columns = ['H_順位'])
    df_tmp15 = pd.DataFrame(a_rank_list, columns = ['A_順位'])
    
    df = pd.concat([df, df_tmp1, df_tmp2, df_tmp3, df_tmp4, df_tmp5, df_tmp6, df_tmp7, df_tmp8, df_tmp9, df_tmp10, df_tmp11, df_tmp12, df_tmp13, df_tmp14, df_tmp15],axis=1)

    drop_colmuns = []
    for i in range(1, 12):
        drop_colmuns += ['H_ポジション' + str(i)]
        drop_colmuns += ['A_ポジション' + str(i)]
        drop_colmuns += ['H_選手' + str(i)]
        drop_colmuns += ['A_選手' + str(i)]
        
    df = team_to_index(df, 'H_Team')
    df = team_to_index(df, 'A_Team')
    df = coach_to_index(df)
    
    df.drop(columns = ['天候', 'カテゴリ', 'H_順位', 'A_順位'] + drop_colmuns, inplace = True)
    df.drop(columns = ['スタジアム', '入場者数', '気温', '湿度', '晴_flg', '雨_flg', '曇_flg', '室内_flg', 'キックオフ時'], inplace = True)
    
#     df1 = df['train_test']
#     df.drop(columns = ['train_test'], inplace = True)
    df = df.astype('float64')
#     df = pd.concat([df, df1], axis = 1)
    
    category_columns = ['H_Team', 'A_Team', 'H_監督', 'A_監督']
    for column in category_columns:
        df[column] = pd.Series(df[column]).astype('category')
        
#     df.to_csv("data/model/preprocessing.csv")
    
    return df

def team_to_index(df, tar_col):
    
    M_TEAM_NAMES = pd.read_csv('data/other/TEAM_NAMES.csv').values.tolist()
    
    df = df.copy()
    m_team_list = []
    for i, team_list in enumerate(M_TEAM_NAMES):
        m_team_list += [team_list[0]]

    team_list = []
    for i, row in df.iterrows():
        team_list += [m_team_list.index(row[tar_col])]
    df_tmp = pd.DataFrame(team_list, columns = [tar_col])
    df.drop(columns = [tar_col], inplace=True)
    df = pd.concat([df, df_tmp], axis=1)
    
    return df

def coach_to_index(df):
    M_COACH_LIST = pd.read_csv('data/other/COACH_LIST.csv', index_col=0).reset_index(drop = True).T.values[0].tolist()
    
    for i, row in df.iterrows():
        try:
            M_COACH_LIST.index(row['H_監督'])
        except ValueError:
            M_COACH_LIST += [row['H_監督']]
        try:
            M_COACH_LIST.index(row['A_監督'])
        except ValueError:
            M_COACH_LIST += [row['A_監督']]
    df_corch = pd.DataFrame(M_COACH_LIST, columns = ['監督名'])
#     df_corch.to_csv("data/other/COACH_LIST.csv")
    
    df = df.copy()
    h_coach_list = []
    a_coach_list = []
    for i, row in df.iterrows():
        h_coach_list += [M_COACH_LIST.index(row['H_監督'])]
        a_coach_list += [M_COACH_LIST.index(row['A_監督'])]
            
    df.drop(columns = ['H_監督', 'A_監督'], inplace=True)
    df_tmp1 = pd.DataFrame(h_coach_list, columns = ['H_監督'])
    df_tmp2 = pd.DataFrame(a_coach_list, columns = ['A_監督'])
    df = pd.concat([df, df_tmp1, df_tmp2], axis=1)
    
    return df

In [6]:
def get_y_pred(y_pred_, y_test):
    
    best_accuracy = 0
    thresholds = np.arange(0, 4, 0.01)
    best_threshold = 0
    y_pred = []
    for threshold in thresholds:
        y_pred_tmp = []
        for y in y_pred_:
            if abs(y)<threshold:
                y_pred_tmp += [0]
            elif y > 0:
                y_pred_tmp += [1]
            else:
                y_pred_tmp += [2]
                
        accuracy = sum(y_test == y_pred_tmp) / len(y_test)
        
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            y_pred = y_pred_tmp
            best_threshold = threshold
    
    return np.array(y_pred), best_threshold

In [7]:
def fit_reg(x_train, x_val, x_test, y_train, y_val, y_test):

        lgb_train = lgb_o.Dataset(x_train, y_train)
        lgb_eval = lgb_o.Dataset(x_val, y_val) 
        # LightGBM parameters
        params = {
            'objective': 'regression', # 回帰  
            'metric': 'rmse', # rsme(平均二乗誤差の平方根) 
        }
        # モデルの学習
        model = lgb_o.train(params,
                          train_set=lgb_train,
                          valid_sets=lgb_eval,
                          early_stopping_rounds=100,
                          verbose_eval=200,
                          )
        
        # テストデータの予測
        y_pred = model.predict(x_test, num_iteration=model.best_iteration)
        # rmse : 平均二乗誤差の平方根
        mse = mean_squared_error(y_test, y_pred) # MSE(平均二乗誤差)の算出
        rmse = np.sqrt(mse) # RSME = √MSEの算出
        print('goal_deff rmse : ', rmse)
        
        return y_pred

In [8]:
from sklearn import metrics

def fit_binary(x_train, x_val, x_test, y_train, y_val, y_test):

        lgb_train = lgb_o.Dataset(x_train, y_train)
        lgb_eval = lgb_o.Dataset(x_val, y_val) 
        # 学習用パラメータ
        lgbm_params = {
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'num_class': 2,
        }
        # 学習
        gbm_o = lgb_o.train(lgbm_params,
                        lgb_train,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=100,
                        verbose_eval=200,)

        y_pred_proba = gbm_o.predict(x_test, num_iteration=gbm_o.best_iteration)
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        # Accuracy の計算
        accuracy = sum(y_test == y_pred) / len(y_test)
        print('accuracy:', accuracy)
        
        return y_pred, y_pred_proba

In [9]:
y_columns = ['H_goal', 'A_goal', 'goal_difference', 'H_result']
df = preprocessing()

/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (13,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
train, val = train_test_split(df, train_size=0.75)
val, test = train_test_split(val, train_size=0.75)

y_train = train['goal_difference']
x_train = train.drop(columns = y_columns)
y_val = val['goal_difference']
x_val = val.drop(columns = y_columns)
y_test = test['goal_difference']
x_test = test.drop(columns = y_columns)

In [11]:
y_pred = fit_reg(x_train, x_val, x_test, y_train, y_val, y_test)

[I 2021-05-09 16:12:50,062] A new study created in memory with name: no-name-47fb9449-8ae8-478e-b8c9-8588e174d91b
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001976 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7637
[LightGBM] [Info] Number of data points in the train set: 6750, number of used features: 85
[LightGBM] [Info] Start training from score 0.158815
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 1.171253:  14%|#4        | 1/7 [00:00<00:02,  2.30it/s]

[200]	valid_0's rmse: 1.17446
Early stopping, best iteration is:
[112]	valid_0's rmse: 1.17125
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7637
[LightGBM] [Info] Number of data points in the train set: 6750, number of used features: 85
[LightGBM] [Info] Start training from score 0.158815
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 1.165773:  29%|##8       | 2/7 [00:01<00:03,  1.38it/s]

...

[200]	valid_0's rmse: 1.17453
Early stopping, best iteration is:
[105]	valid_0's rmse: 1.16948
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7637
[LightGBM] [Info] Number of data points in the train set: 6750, number of used features: 85
[LightGBM] [Info] Start training from score 0.158815
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 1.155674:  80%|########  | 4/5 [00:01<00:00,  2.24it/s]

[200]	valid_0's rmse: 1.16575
Early stopping, best iteration is:
[181]	valid_0's rmse: 1.16267
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7637
[LightGBM] [Info] Number of data points in the train set: 6750, number of used features: 85
[LightGBM] [Info] Start training from score 0.158815
Training until validation scores don't improve for 100 rounds
[200]	valid_0's rmse: 1.17798


min_data_in_leaf, val_score: 1.155674: 100%|##########| 5/5 [00:02<00:00,  1.85it/s]

Early stopping, best iteration is:
[159]	valid_0's rmse: 1.17448
goal_deff rmse :  1.1654429651206162


In [12]:
y_pred_, _ = get_y_pred(y_pred, y_test.values)

pred = pd.DataFrame([y_test.values.tolist(), y_pred_, y_pred]).T
pred.columns =  ['y', 'pred', 'pred2']
pred.sort_values('pred')
pred = pred[pred['y'] != 0]
accuracy = sum(test['H_result'].values == y_pred_) / len(y_test)
accuracy

0.5737122557726465

In [13]:
pred

,y,pred,pred2
1,-3.0,2.0,-1.451411
2,-2.0,0.0,-0.591272
5,-3.0,0.0,-0.683611
6,-1.0,0.0,-0.814583
7,2.0,1.0,1.324390
8,-1.0,2.0,-1.272423
9,1.0,1.0,1.260734
11,-1.0,2.0,-1.702133
15,1.0,0.0,0.235838
16,3.0,1.0,2.387550


In [14]:
drop_columns = ['H_シュート', 'H_枠内シュート', 'H_PKによるシュート',
       'H_直接ＦＫ', 'H_間接ＦＫ', 'H_ＣＫ', 'H_クリア', 'H_インターセプト', 'H_オフサイド', 'H_警告',
       'H_退場', 'H_３０ｍライン進入', 'H_ペナルティエリア進入', 'H_攻撃回数', 'H_チャンス構築率', 'H_ボール支配率',
       'A_シュート', 'A_枠内シュート', 'A_PKによるシュート', 'A_直接ＦＫ', 'A_間接ＦＫ', 'A_ＣＫ',
       'A_クリア', 'A_インターセプト', 'A_オフサイド', 'A_警告', 'A_退場', 'A_３０ｍライン進入',
       'A_ペナルティエリア進入', 'A_攻撃回数', 'A_チャンス構築率', 'A_ボール支配率', 'H_成功したパス', 'H_失敗したパス', 'H_成功したクロス',
       'H_失敗したクロス', 'H_成功したスローイン', 'H_失敗したスローイン', 'H_成功したドリブル', 'H_失敗したドリブル',
       'H_成功したタックル', 'H_失敗したタックル', 'A_成功したパス', 'A_失敗したパス', 'A_成功したクロス',
       'A_失敗したクロス', 'A_成功したスローイン', 'A_失敗したスローイン', 'A_成功したドリブル', 'A_失敗したドリブル',
       'A_成功したタックル', 'A_失敗したタックル']

In [15]:
df = preprocessing()

df.loc[df['H_result'] == 0,  'even_flg'] = 0
df.loc[df['H_result'] != 0,  'even_flg'] = 1

df_0 = df[df['even_flg'] == 0]
df_1 = df[df['even_flg'] == 1]
df_1 = df_1.iloc[:df_0.shape[0]]
df = pd.concat([df_0, df_1])
df.drop(columns = drop_columns, inplace = True)

train, val = train_test_split(df, train_size=0.75)
val, test = train_test_split(val, train_size=0.75)

y_train = train['even_flg']
x_train = train.drop(columns = y_columns + ['even_flg'])
y_val = val['even_flg']
x_val = val.drop(columns = y_columns+ ['even_flg'])
y_test = test['even_flg']
x_test = test.drop(columns = y_columns+ ['even_flg'])

y_pred, y_pred_proba = fit_binary(x_train, x_val, x_test, y_train, y_val, y_test)

/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (13,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [16]:
df1 = pd.DataFrame(y_test.values, columns = ['y_test'])
df2 = pd.DataFrame(y_pred, columns = ['y_pred'])
df3 = pd.DataFrame(y_pred_proba, columns = ['proba_0', 'proba_1'])
pred = pd.concat([df1, df2, df3], axis = 1)
pred[pred['y_test'] == 0]

[I 2021-05-09 16:13:38,899] A new study created in memory with name: no-name-f4215634-6f44-4475-bd3b-5a55486f9625
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000869 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 3429, number of used features: 33
[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0.688782
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.372614:  14%|#4        | 1/7 [00:00<00:01,  3.27it/s]

Early stopping, best iteration is:
[36]	valid_0's multi_logloss: 0.372614
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000707 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 3429, number of used features: 33
[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0.688782
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.372614:  29%|##8       | 2/7 [00:00<00:01,  3.29it/s]

...

[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0.688782
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid_0's multi_logloss: 0.356704
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000538 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 3429, number of used features: 33
[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0.688782
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 0.356704:  80%|########  | 4/5 [00:00<00:00,  7.17it/s]

Early stopping, best iteration is:
[78]	valid_0's multi_logloss: 0.356704
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 3429, number of used features: 33
[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0.688782
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid_0's multi_logloss: 0.356704
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000705 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 3429, number of used features: 33
[LightGBM] [Info] Start training from score -0.697531
[LightGBM] [Info] Start training from score -0

min_data_in_leaf, val_score: 0.356704: 100%|##########| 5/5 [00:00<00:00,  6.39it/s]

Early stopping, best iteration is:
[78]	valid_0's multi_logloss: 0.356704
accuracy: 0.8286713286713286


In [36]:
df = preprocessing()
df = df[df['H_result'] != 0]

df.loc[df['H_result'] == 1,  'H_result'] = 0
df.loc[df['H_result'] == 2,  'H_result'] = 1

df_0 = df[df['H_result'] == 0]
df_1 = df[df['H_result'] == 1]
df_0 = df_0.iloc[:df_1.shape[0]]
df = pd.concat([df_0, df_1])
df.drop(columns = drop_columns, inplace = True)

train, val = train_test_split(df, train_size=0.75)
val, test = train_test_split(val, train_size=0.75)

y_train = train['H_result']
x_train = train.drop(columns = y_columns)
y_val = val['H_result']
x_val = val.drop(columns = y_columns)
y_test = test['H_result']
x_test = test.drop(columns = y_columns)

y_pred, y_pred_proba = fit_binary(x_train, x_val, x_test, y_train, y_val, y_test)

/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (13,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
[I 2021-05-09 16:23:12,856] A new study created in memory with name: no-name-15188812-fbe2-4fb2-a7c5-de708e450b3e
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000859 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1761
[LightGBM] [Info] Number of data points in the train set: 4600, number of used features: 33
[LightGBM] [Info] Start training from score -0.700128
[LightGBM] [Info] Start training from score -0.686215
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.617314:  14%|#4        | 1/7 [00:00<00:02,  2.96it/s]

Early stopping, best iteration is:
[22]	valid_0's multi_logloss: 0.617314
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000926 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1761
[LightGBM] [Info] Number of data points in the train set: 4600, number of used features: 33
[LightGBM] [Info] Start training from score -0.700128
[LightGBM] [Info] Start training from score -0.686215
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.617164:  29%|##8       | 2/7 [00:00<00:01,  3.25it/s]

...

Early stopping, best iteration is:
[30]	valid_0's multi_logloss: 0.611963
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000960 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1761
[LightGBM] [Info] Number of data points in the train set: 4600, number of used features: 33
[LightGBM] [Info] Start training from score -0.700128
[LightGBM] [Info] Start training from score -0.686215
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 0.609350:  80%|########  | 4/5 [00:01<00:00,  2.71it/s]

Early stopping, best iteration is:
[33]	valid_0's multi_logloss: 0.617909
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000533 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1761
[LightGBM] [Info] Number of data points in the train set: 4600, number of used features: 33
[LightGBM] [Info] Start training from score -0.700128
[LightGBM] [Info] Start training from score -0.686215
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 0.609350: 100%|##########| 5/5 [00:01<00:00,  2.59it/s]

Early stopping, best iteration is:
[30]	valid_0's multi_logloss: 0.615384
accuracy: 0.6197916666666666


In [39]:
df1 = pd.DataFrame(y_test.values, columns = ['y_test'])
df2 = pd.DataFrame(y_pred, columns = ['y_pred'])
df3 = pd.DataFrame(y_pred_proba, columns = ['proba_0', 'proba_1'])
pred = pd.concat([df1, df2, df3], axis = 1)
pred[pred['y_test'] == 1]

,y_test,y_pred,proba_0,proba_1
0,1.0,1,0.085395,0.914605
5,1.0,1,0.387343,0.612657
8,1.0,1,0.426747,0.573253
9,1.0,1,0.329592,0.670408
10,1.0,1,0.227216,0.772784
11,1.0,0,0.704233,0.295767
16,1.0,0,0.699508,0.300492
17,1.0,0,0.647407,0.352593
18,1.0,1,0.283585,0.716415
19,1.0,0,0.601987,0.398013


In [41]:
x_train.columns

Index(['年月日', '節', 'H_勝点', 'H_試合', 'H_勝', 'H_分', 'H_敗', 'H_得点', 'H_失点',
       'H_得失点差', 'H_rest_days', 'A_勝点', 'A_試合', 'A_勝', 'A_分', 'A_敗', 'A_得点',
       'A_失点', 'A_得失点差', 'A_rest_days', 'J1_flg', 'J2_flg', 'J3_flg', 'H_DF',
       'H_MF', 'H_FW', 'A_DF', 'A_MF', 'A_FW', 'H_Team', 'A_Team', 'H_監督',
       'A_監督'],
      dtype='object')

In [42]:
df = preprocessing()

/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (13,24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [44]:
df[df['H_goal'] > 2]

,年月日,節,H_勝点,H_試合,H_勝,H_分,H_敗,H_得点,H_失点,H_得失点差,H_rest_days,A_勝点,A_試合,A_勝,A_分,A_敗,A_得点,A_失点,A_得失点差,A_rest_days,H_シュート,H_枠内シュート,H_PKによるシュート,H_直接ＦＫ,H_間接ＦＫ,H_ＣＫ,H_クリア,H_インターセプト,H_オフサイド,H_警告,H_退場,H_３０ｍライン進入,H_ペナルティエリア進入,H_攻撃回数,H_チャンス構築率,H_ボール支配率,A_シュート,A_枠内シュート,A_PKによるシュート,A_直接ＦＫ,A_間接ＦＫ,A_ＣＫ,A_クリア,A_インターセプト,A_オフサイド,A_警告,A_退場,A_３０ｍライン進入,A_ペナルティエリア進入,A_攻撃回数,A_チャンス構築率,A_ボール支配率,H_goal,A_goal,goal_difference,H_result,H_成功したパス,H_失敗したパス,H_成功したクロス,H_失敗したクロス,H_成功したスローイン,H_失敗したスローイン,H_成功したドリブル,H_失敗したドリブル,H_成功したタックル,H_失敗したタックル,A_成功したパス,A_失敗したパス,A_成功したクロス,A_失敗したクロス,A_成功したスローイン,A_失敗したスローイン,A_成功したドリブル,A_失敗したドリブル,A_成功したタックル,A_失敗したタックル,J1_flg,J2_flg,J3_flg,H_DF,H_MF,H_FW,A_DF,A_MF,A_FW,H_Team,A_Team,H_監督,A_監督
17,20210425.0,11.0,14.0,10.0,4.0,2.0,4.0,8.0,13.0,-5.0,4.0,5.0,9.0,1.0,2.0,6.0,5.0,14.0,-9.0,4.0,9.0,4.0,0.0,12.0,0.0,4.0,18.0,3.0,0.0,0.0,0.0,45.0,16.0,113.0,8.0,63.5,16.0,6.0,0.0,4.0,0.0,6.0,22.0,4.0,0.0,1.0,0.0,41.0,11.0,108.0,14.8,36.5,3.0,2.0,1.0,1.0,670.770,100.230,4.997,14.003,29.000,0.000,5.004,6.996,13.005,1.995,334.836,91.164,2.002,11.998,11.004,0.996,1.000,3.000,18.009,4.991,1.0,0.0,0.0,4.0,4.0,2.0,4.0,5.0,1.0,3.0,19.0,5.0,43.0
20,20210424.0,11.0,18.0,9.0,5.0,3.0,1.0,15.0,8.0,7.0,3.0,2.0,10.0,0.0,2.0,8.0,7.0,27.0,-20.0,3.0,24.0,7.0,1.0,8.0,4.0,7.0,19.0,4.0,5.0,0.0,0.0,65.0,23.0,124.0,19.4,60.1,7.0,4.0,0.0,9.0,5.0,8.0,26.0,2.0,4.0,1.0,0.0,21.0,7.0,118.0,5.9,39.9,5.0,0.0,5.0,1.0,619.875,105.125,7.002,10.998,21.000,3.000,4.998,2.002,8.993,8.007,277.140,94.860,0.000,13.000,15.008,0.992,10.005,4.995,5.008,10.992,1.0,0.0,0.0,4.0,3.0,3.0,4.0,4.0,2.0,7.0,8.0,51.0,127.0
24,20210424.0,11.0,10.0,10.0,3.0,1.0,6.0,7.0,12.0,-5.0,3.0,14.0,11.0,4.0,2.0,5.0,11.0,11.0,0.0,3.0,9.0,5.0,0.0,15.0,1.0,3.0,18.0,2.0,2.0,1.0,0.0,24.0,7.0,120.0,7.5,34.3,10.0,2.0,0.0,13.0,2.0,1.0,8.0,0.0,1.0,1.0,0.0,34.0,7.0,118.0,8.5,65.7,5.0,1.0,4.0,1.0,248.092,89.908,2.000,3.000,16.008,7.992,1.002,4.998,16.000,0.000,578.160,81.840,1.002,4.998,22.008,1.992,7.000,3.000,10.000,6.000,1.0,0.0,0.0,3.0,4.0,3.0,3.0,4.0,3.0,4.0,16.0,44.0,229.0
27,20210421.0,9.0,11.0,8.0,3.0,2.0,3.0,10.0,9.0,1.0,4.0,22.0,8.0,7.0,1.0,0.0,14.0,3.0,11.0,4.0,16.0,6.0,0.0,13.0,1.0,3.0,27.0,2.0,0.0,1.0,0.0,41.0,9.0,110.0,14.5,44.9,8.0,3.0,0.0,7.0,0.0,5.0,12.0,1.0,1.0,1.0,0.0,54.0,8.0,109.0,7.3,55.1,3.0,0.0,3.0,1.0,371.917,97.083,5.004,6.996,15.000,0.000,5.000,5.000,7.000,3.000,582.787,120.213,3.002,15.998,15.997,1.003,3.000,2.000,14.007,6.993,0.0,1.0,0.0,4.0,5.0,1.0,4.0,5.0,1.0,28.0,41.0,27.0,26.0
33,20210421.0,9.0,6.0,8.0,1.0,3.0,4.0,7.0,12.0,-5.0,4.0,7.0,8.0,1.0,4.0,3.0,8.0,13.0,-5.0,4.0,17.0,7.0,1.0,8.0,0.0,4.0,14.0,3.0,2.0,0.0,0.0,39.0,10.0,111.0,15.3,46.0,8.0,1.0,0.0,5.0,2.0,3.0,19.0,2.0,0.0,1.0,0.0,37.0,3.0,108.0,7.4,54.0,3.0,1.0,2.0,1.0,397.794,103.206,4.000,12.000,9.000,11.000,6.003,2.997,10.005,4.995,491.063,115.937,0.000,13.000,19.000,1.000,4.000,4.000,5.000,3.000,0.0,1.0,0.0,4.0,3.0,3.0,4.0,4.0,2.0,38.0,31.0,0.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8962,20120317.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,7.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,7.0,12.0,3.0,0.0,14.0,1.0,6.0,33.0,0.0,1.0,1.0,0.0,46.0,8.0,143.0,8.4,56.1,10.0,4.0,0.0,23.0,1.0,3.0,26.0,4.0,1.0,3.0,1.0,41.0,14.0,150.0,6.7,43.9,3.0,2.0,1.0,1.0,362.880,141.120,2.000,6.000,19.992,1.008,7.000,7.000,22.008,1.992,246.888,134.112,4.004,9.996,31.010,3.990,3.003,7.997,14.007,6.993,1.0,0.0,0.0,4.0,5.0,1.0,4.0,3.0,3.0,5.0,11.0,27.0,197.0
8966,20120317.0,3.0,3.0,2.0,1.0,0.0,1.0,2.0,3.0,-1.0,6.0,3.0,2.0,1.0,0.0,1.0,5.0,3.0,2.0,6.0,19.0,7.0,1.0,9.0,3.0,7.0,28.0,1.0,2.0,2.0,0.0,43.0,16.0,144.0,13.2,49.6,12.0,4.0,0.0,19.0,2.0,4.0,28.0,1.0,3.0,3.0,0.0,30.0,13.0,147.0,8.2,50.4,3.0,2.0,1.0,1.0,293.970,116.030,4.005,10.995,25.004,2.996,6.006